In [ ]:
import os
import glob
import pandas as pd

In [ ]:
csv_files = glob.glob("one_hidden_mlp_records/*/*trials.csv")

In [ ]:
def make_dict(file_list):
    data_dict = {}
    for f in file_list:
        # Extract the middle portion: records/<key>/<file>
        key = os.path.basename(os.path.dirname(f))  # <--- folder name
        if "parzen_sampler" in key:
            data_dict[key.replace("_parzen_sampler", "")] = pd.read_csv(f)
        elif "grid_sampler" in key:
            data_dict[key.replace("_grid_sampler", "")] = pd.read_csv(f)
        elif "random_sampler" in key:
            data_dict[key.replace("_random_sampler", "")] = pd.read_csv(f)
    return data_dict

In [ ]:
parzen_files = [f for f in csv_files if "parzen_sampler" in f]
grid_files   = [f for f in csv_files if "grid_sampler" in f]
random_files = [f for f in csv_files if "random_sampler" in f]
parzen_dict = make_dict(parzen_files)
grid_dict   = make_dict(grid_files)
random_dict = make_dict(random_files)

In [ ]:
from vizs import plot_efficiency_curves_best_of_N, plot_ard_panels, plot_accuracy_vs_param_panels

In [ ]:
plot_efficiency_curves_best_of_N(parzen_dict, grid_dict, random_dict, exp_ks=(1,2,4,8,16,32))

In [ ]:
plot_ard_panels(parzen_dict, grid_dict, random_dict, n_boot=64)

In [ ]:
fig, corr = plot_accuracy_vs_param_panels(grid_dict, grid_dict, grid_dict,
                                           acc_col="value", param_col="value", plot_mode="scatter",
                                           save_path="acc_vs_param_8panels_scatter.png")

In [ ]:
fig, corr = plot_accuracy_vs_param_panels(grid_dict, grid_dict, grid_dict,
                                           acc_col="value", param_col="value", plot_mode="bland_altman",
                                           save_path="acc_vs_param_8panels_balt.png")